### developing chatbot plan : 

the model will be simple llm model . i will use "Attention is what you need" paper's diagram to create the model
from scratch.. 

In [35]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import re
import string
import json
import os
from module.createData import CreateDataset
from module.readData import ReadDataset
from module.cuda import getDevice
from module.bot import Chatbot
from module.Transformer import Transformer
from module.TrainModel import trainModel
from module.TestModel import Test

from module.VD import VectorDatabase

In [36]:
with open('asset/data/data.txt', 'r',encoding='utf-8') as f:
    data = f.read()

In [37]:
data

"1.2 \nFINANCIAL POLICY OBJECTIVES AND STRATEGIES \n STATEMENT \nThe presentation and preparation of the Territory’s Budget is provided for in sections 11 and \n11A of the Financial Management Act 1996 (the Act).   \nThe purpose of the financial policy objectives and strategies statement is to make transparent \nthe Government’s financial strategies and to establish a benchmark for evaluating the \nGovernment’s conduct of financial policy.  The statement is also consistent with section \n11(1)(a) of the Act. \nStrategic Priorities and Financial Policy \nIn this budget, the Government continues its commitment to the principles of responsible \nfinancial management. \nThe financial objectives and the key measures below outline the Government’s financial \npolicy.  Strategic priorities, as they relate to the Territory’s Budget, are summarised as: \n• maintain a balanced budget over the economic cycle; \n• maintain low levels of debt; \n• provide the highest possible standard of government

In [38]:
getDevice = getDevice()

In [39]:
def ExtractKeyInfo(InputFilePath, OutputFilePath):
    if not os.path.exists(InputFilePath):
        raise FileNotFoundError(f"Input file {InputFilePath} not found.")
    
    with open(InputFilePath, 'r', encoding='utf-8') as F:
        Document = F.read()
    
    Pages = re.split(r'2005-06 Budget Paper No. 3\s*(\d+)\s*Financial Policy Objectives and Strategies Statement', Document)
    PageContents = []
    CurrentPage = 5
    for I in range(0, len(Pages), 2):
        Content = Pages[I].strip()
        if Content:
            PageContents.append({'Page': CurrentPage, 'Content': Content})
        if I+1 < len(Pages):
            CurrentPage = int(Pages[I+1])
    
    KeyInfo = []
    Strategic = re.search(r'Strategic priorities, as they relate to the Territory’s Budget, are summarised as:(.*?)\. The 2005-06 Budget and Forward Estimates', Document, re.S)
    if Strategic:
        Priorities = re.findall(r'• (.*?);', Strategic.group(1))
        for P in Priorities:
            KeyInfo.append({'Info': P.strip(), 'Source': f'Page {CurrentPage}'})
    
    KeyInfo.extend([
        {'Info': 'The Budget provides an aggregate surplus over four years. Cash reserves reducing through to 2006-07 but forecast to grow in 2008-09, general government borrowings not increased.', 'Source': 'Page 5'},
        {'Info': 'Resources directed into health, education, child and family support, meeting demand of growing and ageing population, targeted at vulnerable.', 'Source': 'Page 5'},
        {'Info': 'Investments in Shaping Our Territory plan, ACT Energy Wise Program, Asbestos Assessment Task Force.', 'Source': 'Page 5'},
        {'Info': 'Financial Objectives and Measures (Table 1.2.1): Short Term: Maintain Triple A credit rating, low debt, infrastructure, long-term liabilities, strategic capital works, minimise risk, high standard services. Long Term: Balanced budget, 90% superannuation coverage by 2039-2040, reduce GGS debt, net interest cost <0, mitigate risk, taxation % GSP, increase net assets.', 'Source': 'Page 6'},
        {'Info': 'GGS Operating Result (Table 1.2.2): 2002-03 154.6m, 2003-04 70.5m, 2004-05 -52.2m, 2005-06 91.5m, 2006-07 0.9m, 2007-08 39.3m, 2008-09 73.3m. Aggregate 2002-03 to 2005-06 185.7m, 2005-06 to 2008-09 22.0m.', 'Source': 'Page 6'},
        {'Info': 'Capital Infrastructure (Table 1.2.3): Non-Current WIP 136m to 41m, Property Plant Equipment 9,905m to 11,034m, Sub Total 10,040m to 11,075m.', 'Source': 'Page 6'},
        {'Info': 'Net Interest Cost % Own-Source Revenue (Table 1.2.4): -1.9% to -0.1%.', 'Source': 'Page 7'},
        {'Info': 'Taxation % GSP (Table 1.2.5): 4.1% to 4.4%.', 'Source': 'Page 8'},
        {'Info': 'Total Territory Net Assets (Table 1.2.6): 9,304m to 9,878m.', 'Source': 'Page 8'},
        {'Info': 'Superannuation Liabilities Funding (Table 1.2.7): 58% in 2005 to 67% in 2009.', 'Source': 'Page 9'},
        {'Info': 'Principles of Responsible Financial Management: Prudent liabilities, balanced expenses, net worth buffer, fiscal risks, stable taxing, full disclosure.', 'Source': 'Page 9'},
        {'Info': 'Infrastructure investments: Stromlo Forest Park, Primary and Pre-School in East Gungahlin, Quamby replacement, Gungahlin Drive Extension, Alexander Maconachie Centre.', 'Source': 'Page 10'},
    ])
    
    with open(OutputFilePath, 'w', encoding='utf-8') as F:
        json.dump(KeyInfo, F, indent=4)
    
    return KeyInfo


In [40]:
def GenerateDatasetFile(KeyInfo, DatasetFilePath):
    ansPair = [
        ("What is the budget strategy?", "The Budget provides an aggregate surplus over four years. Cash reserves reducing through to 2006-07 but forecast to grow in 2008-09, general government borrowings not increased. (Source: Page 5)"),
        ("What about debt levels?", "Financial Objectives and Measures (Table 1.2.1): Short Term: Maintain Triple A credit rating, low debt, infrastructure, long-term liabilities, strategic capital works, minimise risk, high standard services. Long Term: Balanced budget, 90% superannuation coverage by 2039-2040, reduce GGS debt, net interest cost <0, mitigate risk, taxation % GSP, increase net assets. (Source: Page 6)"),
        ("What are the infrastructure investments?", "Infrastructure investments: Stromlo Forest Park, Primary and Pre-School in East Gungahlin, Quamby replacement, Gungahlin Drive Extension, Alexander Maconachie Centre. (Source: Page 10)"),
        ("What is the net interest cost?", "Net Interest Cost % Own-Source Revenue (Table 1.2.4): -1.9% to -0.1%. (Source: Page 7)"),
        ("How is taxation managed?", "Taxation % GSP (Table 1.2.5): 4.1% to 4.4%. (Source: Page 8)"),
        ("What about superannuation liabilities?", "Superannuation Liabilities Funding (Table 1.2.7): 58% in 2005 to 67% in 2009. (Source: Page 9)"),
        ("What are the principles of financial management?", "Principles of Responsible Financial Management: Prudent liabilities, balanced expenses, net worth buffer, fiscal risks, stable taxing, full disclosure. (Source: Page 9)"),
        ("What are the strategic priorities?", "maintain a balanced budget over the economic cycle (Source: Page 5)"),
        ("What are the services prioritized?", "Resources directed into health, education, child and family support, meeting demand of growing and ageing population, targeted at vulnerable. (Source: Page 5)"),
        ("What are the investment programs?", "Investments in Shaping Our Territory plan, ACT Energy Wise Program, Asbestos Assessment Task Force. (Source: Page 5)")
    ]
    
    with open(DatasetFilePath, 'w', encoding='utf-8') as F:
        for Question, Answer in ansPair:
            F.write(f'"{Question}", "{Answer}"\n')
    
    return ansPair

### Datapath initialize

In [41]:
InputFilePath = 'asset/data/data.txt'
JsonFilePath = 'asset/preprocessed/extractedData.json'
DatasetFilePath = 'asset/preprocessed/dataset.txt'

In [42]:
if not os.path.exists(JsonFilePath):
    KeyInfo = ExtractKeyInfo(InputFilePath, JsonFilePath)
else:
    with open(JsonFilePath, 'r', encoding='utf-8') as F:
        KeyInfo = json.load(F)


if not os.path.exists(DatasetFilePath):
    GenerateDatasetFile(KeyInfo, DatasetFilePath)

### Initializing vector DB

In [43]:
Db = VectorDatabase(KeyInfo)

### vocabulary creation

In [44]:
vocab = set()

for Item in KeyInfo:
    vocab.update(Item['Info'].lower().translate(str.maketrans('', '', string.punctuation)).split())
vocab.update(['<START>', '<END>', '<PAD>'])

lti = {Word: Idx for Idx, Word in enumerate(vocab)}

itl = {Idx: Word for Word, Idx in lti.items()}

In [45]:
print(lti)

{'aggregate': 0, 'by': 1, 'mitigate': 2, '200708': 3, 'balanced': 4, 'wip': 5, 'population': 6, '123': 7, '41': 8, 'investments': 9, '126': 10, 'family': 11, 'park': 12, 'vulnerable': 13, 'capital': 14, 'resources': 15, '125': 16, 'operating': 17, 'child': 18, 'prudent': 19, '11075m': 20, 'maintain': 21, '1857m': 22, 'fiscal': 23, 'over': 24, 'provides': 25, 'gungahlin': 26, 'maconachie': 27, 'not': 28, 'of': 29, 'an': 30, '200607': 31, 'years': 32, 'cash': 33, 'risk': 34, 'ggs': 35, 'minimise': 36, 'four': 37, 'to': 38, '90': 39, 'funding': 40, 'buffer': 41, 'gsp': 42, '127': 43, '<END>': 44, 'cost': 45, 'sub': 46, 'act': 47, 'superannuation': 48, 'long': 49, 'government': 50, 'debt': 51, 'asbestos': 52, '393m': 53, '44': 54, 'in': 55, 'equipment': 56, 'low': 57, 'directed': 58, 'growing': 59, 'territory': 60, 'and': 61, 'plant': 62, '41m': 63, 'responsible': 64, 'ownsource': 65, 'table': 66, '1546m': 67, '522m': 68, '<PAD>': 69, '220m': 70, 'result': 71, 'ageing': 72, '136m': 73, '12

In [46]:
print(itl)

{0: 'aggregate', 1: 'by', 2: 'mitigate', 3: '200708', 4: 'balanced', 5: 'wip', 6: 'population', 7: '123', 8: '41', 9: 'investments', 10: '126', 11: 'family', 12: 'park', 13: 'vulnerable', 14: 'capital', 15: 'resources', 16: '125', 17: 'operating', 18: 'child', 19: 'prudent', 20: '11075m', 21: 'maintain', 22: '1857m', 23: 'fiscal', 24: 'over', 25: 'provides', 26: 'gungahlin', 27: 'maconachie', 28: 'not', 29: 'of', 30: 'an', 31: '200607', 32: 'years', 33: 'cash', 34: 'risk', 35: 'ggs', 36: 'minimise', 37: 'four', 38: 'to', 39: '90', 40: 'funding', 41: 'buffer', 42: 'gsp', 43: '127', 44: '<END>', 45: 'cost', 46: 'sub', 47: 'act', 48: 'superannuation', 49: 'long', 50: 'government', 51: 'debt', 52: 'asbestos', 53: '393m', 54: '44', 55: 'in', 56: 'equipment', 57: 'low', 58: 'directed', 59: 'growing', 60: 'territory', 61: 'and', 62: 'plant', 63: '41m', 64: 'responsible', 65: 'ownsource', 66: 'table', 67: '1546m', 68: '522m', 69: '<PAD>', 70: '220m', 71: 'result', 72: 'ageing', 73: '136m', 74:

### Transformer parameters

In [47]:
DModel = 512
FfnHidden = 2048
NumHeads = 8
DropProb = 0.1
NumLayers = 6
MaxSequenceLength = 200
VocabSize = len(vocab)
StartToken ='<START>'
EndToken ='<END>'
PaddingToken ='<PAD>'

In [48]:
model = Transformer(DModel, FfnHidden, NumHeads, DropProb, NumLayers, MaxSequenceLength, VocabSize, lti, StartToken, EndToken, PaddingToken)
model.to(getDevice)

Transformer(
  (Encoder): Encoder(
    (SentenceEmbedding): SentenceEmbedding(
      (Embedding): Embedding(177, 512)
      (PositionEncoder): PE()
      (Dropout): Dropout(p=0.1, inplace=False)
    )
    (Layers): SequentialEncoder(
      (0): EncoderLayer(
        (Attention): MultiHeadAttention(
          (QkvLayer): Linear(in_features=512, out_features=1536, bias=True)
          (LinearLayer): Linear(in_features=512, out_features=512, bias=True)
        )
        (Norm1): LayerNormalization()
        (Dropout1): Dropout(p=0.1, inplace=False)
        (Ffn): PositionwiseFeedForward(
          (Linear1): Linear(in_features=512, out_features=2048, bias=True)
          (Linear2): Linear(in_features=2048, out_features=512, bias=True)
          (Relu): ReLU()
          (Dropout): Dropout(p=0.1, inplace=False)
        )
        (Norm2): LayerNormalization()
        (Dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): EncoderLayer(
        (Attention): MultiHeadAttention(
          

In [49]:
Dataset = CreateDataset(DatasetFilePath,lti,MaxSequenceLength)

In [50]:
print(Dataset)

[(tensor([ 82,  69,  69,  85, 120,  69,  44,  69,  69,  69,  69,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,

In [51]:
trainModel(model, Dataset,lti,Epochs=10,BatchSize=2,LearningRate=0.0001)

ValueError: Expected input batch_size (400) to match target batch_size (398).

In [ ]:
TestQueries = ["What about debt?","Tell me about infrastructure.","What is the budget strategy?"]
TestResults = Test(model,lti ,itl, MaxSequenceLength, TestQueries)

In [ ]:
for Query, Response in TestResults:
    print(f"Query: {Query}\nResponse: {Response}\n")

## check q/a

In [ ]:
xc = Chatbot(Db,model,lti,itl,MaxSequenceLength,StartToken,EndToken,PaddingToken)

In [ ]:
print(xc.Ask("What about debt?"))


In [ ]:
print(xc.Ask("Tell me about infrastructure."))